In [17]:
from dotenv import load_dotenv
import os 

from langchain_groq.chat_models import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
import bs4

In [ ]:
# url = "https://medium.com/relari/choosing-the-right-ai-agent-framework-langgraph-vs-crewai-vs-openai-swarm-56f7931b4249?source=search_post---------3-----------------------------------"

# docs = WebBaseLoader(url).load()
# docs

[Document(metadata={'source': 'https://medium.com/relari/choosing-the-right-ai-agent-framework-langgraph-vs-crewai-vs-openai-swarm-56f7931b4249?source=search_post---------3-----------------------------------', 'title': 'Choosing the Right AI Agent Framework: LangGraph vs CrewAI vs OpenAI Swarm | Relari | Relari Blog', 'description': 'In-depth LLM Agent Framework comparison of LangGraph, CrewAI, OpenAI Swarm.', 'language': 'en'}, page_content='Choosing the Right AI Agent Framework: LangGraph vs CrewAI vs OpenAI Swarm | Relari | Relari BlogOpen in appSign upSign inWriteSign upSign inChoosing the Right AI Agent Framework: LangGraph vs CrewAI vs OpenAI SwarmYi Zhang·FollowPublished inRelari Blog·15 min read·Dec 2, 2024--16ListenShareIn-depth comparison of agent orchestration with the same Agentic Finance App built using 3 different frameworks.Image generated by ChatGPT. Prompt: Multi-agent collaboration system.What We Will CoverWhat are agents? A deeper look at how we define agents and wha

In [20]:
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        # filter specific parts of the webpage, improving efficienty.
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

In [30]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

In [32]:
print(docs[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
len(documents)

65

In [22]:
documents[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refin

In [23]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [24]:
groq_api_key = os.getenv('GROQ_API_KEY')

llm = ChatGroq(groq_api_key=groq_api_key, model_name="qwen-2.5-32b")
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_time': 0.05, 'prompt_time': 0.003742287, 'queue_time': 0.050864408, 'total_time': 0.053742287}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-711828be-dd00-42b3-a937-c8db3fd21b9f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [25]:
vstore = FAISS.from_documents(documents=documents, embedding=embeddings)
vstore.save_local("faiss_index")

In [26]:
db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()

In [27]:
retriever.invoke("when to use crew ai")

[Document(id='3cfc0463-d691-43a8-ade1-e37c086ec825', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content="(3) Task execution: Expert models execute on the specific tasks and log results.\nInstruction:\n\nWith the input and the inference results, the AI assistant needs to describe the process and results. The previous stages can be formed as - User Input: {{ User Input }}, Task Planning: {{ Tasks }}, Model Selection: {{ Model Assignment }}, Task Execution: {{ Predictions }}. You must first answer the user's request in a straightforward manner. Then describe the task process and show your analysis and model inference results to the user in the first person. If inference results contain a file path, must tell the user the complete file path."),
 Document(id='0f92e937-f75f-407e-b33e-978f78b0cfa4', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 10. A picture of a sea otter using rock to crack open a seash

In [37]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [38]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\sowmy\anaconda3\envs\langchain\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [39]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [40]:
from langchain_core.runnables import RunnablePassthrough

In [41]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [43]:
response = rag_chain.invoke("What is Task Decomposition?")

In [45]:
print(response)

Task decomposition is the process of breaking down a complex task into smaller, more manageable steps to simplify problem-solving. This can be achieved through techniques like Chain of Thought (CoT) and Tree of Thoughts, which guide large language models to think step by step, or through direct prompting and human input.
